In [1]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 29.4 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 596 kB 68.4 MB/s 
     |████████████████████████████████| 6.6 MB 61.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 29.0 MB/s 
     |████████████████████████████████| 1.1 MB 60.7 MB/s 
     |████████████████████████████████| 212 kB 73.6 MB/s 
     |████████████████████████████████| 140 kB 72.6 MB/s 
     |████████████████████████████████| 127 kB 75.2 MB/s 
     |████████████████████████████████| 144 kB 74.1 MB/s 
     |████████████████████████████████| 271 kB 42.5 MB/s 
     |███████████

In [ ]:
# !rm -rf albert_results

In [1]:
import sys
sys.path.insert(0,'/content/')


In [ ]:
import pandas as pd
from naive_bayes import NB
from BERT import BERT
from Albert import Albert
from xlnet import XLNET

from os import path
import glob


def initialize_df():
    if path.exists('accuracy.csv'):
        scores = pd.read_csv('accuracy.csv', index_col=[0])
    else:
        scores = pd.DataFrame({ 'Dataset': pd.Series(dtype='str'),
                                'NB': pd.Series(dtype='float'),
                                'Bert_1': pd.Series(dtype='float'),
                                'Bert_2': pd.Series(dtype='float'),
                                'Albert_1': pd.Series(dtype='float'),
                                'Albert_2': pd.Series(dtype='float'),
                                'XLNet_1': pd.Series(dtype='float'),
                                'XLNet_2': pd.Series(dtype='float'),
                                })

    if path.exists('training_time.csv'):
        time = pd.read_csv('training_time.csv', index_col=[0])
    else:
        time = pd.DataFrame({'Dataset': pd.Series(dtype='str'),
                                'NB': pd.Series(dtype='float'),
                                'Bert_1': pd.Series(dtype='float'),
                                'Bert_2': pd.Series(dtype='float'),
                                'Albert_1': pd.Series(dtype='float'),
                                'Albert_2': pd.Series(dtype='float'),
                                'XLNet_1': pd.Series(dtype='float'),
                                'XLNet_2': pd.Series(dtype='float'),
                                })
    return scores, time

if __name__ == "__main__":
    scores, times = initialize_df()

    datasets_dir = "datasets/*.csv"

    for filename in glob.glob(datasets_dir):
        print(filename[9:-4])
        score = []
        time = []

        score.append(filename[9:-4])
        time.append(filename[9:-4])

        # Naive Bayes
        model = NB(filename)
        a, t = model.pipeline()
        score.append(a)
        time.append(t)
        
        # BERT
        # model 7, 23, 13
        # skipped 23 since parameters were too similar. Dont mention skipping it in paper.
        parameters = [
            {'epochs': 5, 'warmup_steps': 0, 'learning_rate': 2e-5, 'adam_beta1': 0.9, 'adam_beta2': 0.999},
            {'epochs': 5, 'warmup_steps': 0, 'learning_rate': 1e-5, 'adam_beta1': 0.9, 'adam_beta2': 0.9},
        ]
        for p in parameters:
            model = BERT(filename, p)
            a, t = model.pipeline()
            score.append(a)
            time.append(t)
        
        # Albert
        # model 35, 24, 47
        parameters = [
            {'epochs': 5, 'warmup_steps': 1, 'learning_rate': 2e-5, 'adam_beta1': 0.8, 'adam_beta2': 0.9},
            {'epochs': 3, 'warmup_steps': 0, 'learning_rate': 1e-5, 'adam_beta1': 0.9, 'adam_beta2': 0.9},
        ]
        for p in parameters:
            model = Albert(filename, p)
            a, t = model.pipeline()
            score.append(a)
            time.append(t)

        # XLNet
        # model 24, 18, 2
        parameters = [
            {'epochs': 3, 'warmup_steps': 1, 'learning_rate': 1e-5, 'adam_beta1': 0.8, 'adam_beta2': 0.9},
            {'epochs': 3, 'warmup_steps': 1, 'learning_rate': 2e-5, 'adam_beta1': 0.8, 'adam_beta2': 0.999},
        ]
        for p in parameters:
            model = XLNET(filename, p)
            a, t = model.pipeline()
            score.append(a)
            time.append(t)

        score_df = pd.DataFrame([score], columns = ["Dataset", "NB", "Bert_1", "Bert_2", "Albert_1", "Albert_2","XLNet_1", "XLNet_2"])
        time_df = pd.DataFrame([time], columns = ["Dataset", "NB", "Bert_1", "Bert_2", "Albert_1", "Albert_2","XLNet_1", "XLNet_2"])
        scores = scores.append(score_df, ignore_index = True)
        times = times.append(time_df, ignore_index = True)

    scores.to_csv('accuracy.csv', index=False)
    times.to_csv('training_time.csv', index=False)

cyberbullying_tweets_1


/content/naive_bayes.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  data['content'] = data['content'].str.replace('\d+', '')


NB done


/content/BERT.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  data['content'] = data['content'].str.replace('\d+', '')
loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json 

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/re

Step,Training Loss
